In [1]:
import sys

import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import importlib
from scipy.io import FortranFile

import time 
import cftime 

# import modules in other directories
sys.path.append('../Utils/')
import GridUtils as GrU
import MakePressures as MkP
import humiditycalcs as hum
import MyConstants as Con

sys.path.append('../Regridder/')
import Initialize as Prep
import ReadInSrc as RdSrc
import WriteDST as WrDst
import GenRegrid as GR


importlib.reload( GR )
importlib.reload( Prep )
importlib.reload( RdSrc )
importlib.reload( WrDst )


Using Flexible parallel/serial VertRegrid 
Using Flexible parallel/serial VertRegrid 


<module 'WriteDST' from '/glade/work/juliob/PyRegridding/Drivers/../Regridder/WriteDST.py'>

## START HERE FOR "BATCH"

In [2]:
RegridMethod="CONSERVE" 
foo = Prep.prep(Dst='Arctic',DstVgrid='L32',Src='ERA5',WOsrf=True, RegridMethod=RegridMethod )

In prep Src= ERA5 to Dst=Arctic 
Generating regridding weights. Method CONSERVE : ESMF method= 2
 Src scripfile /glade/work/juliob/ERA5-proc/ERA5interp/grids/ERA5_640x1280_scrip.nc 
 Dst scripfile /glade/work/aherring/grids/var-res/ne0np4.ARCTIC.ne30x4/grids/ne0ARCTICne30x4_scrip_191212.nc 
 Src topo file /glade/work/juliob/ERA5-proc/ERA5interp/phis/ERA5_phis.nc 
 Dst topo file /glade/p/cesmdata/cseg/inputdata/atm/cam/topo/se/ne30x4_ARCTIC_nc3000_Co060_Fi001_MulG_PF_RR_Nsw042_c200428.nc 
 L32 Dst vertical grid from /glade/p/cesmdata/cseg/inputdata/atm/cam/inic/se/f.e22.FC2010climo.ne30pg3_ne30pg3_mg17.cam6_2_022.002.cam.i.0020-01-01-00000_c200610.nc 
Prepping for ERA5 to Arctic proc in Initialize took  36.8517 seconds


In [3]:
faa =RdSrc.get_Src( year=2021,month=6,day=1 , hour0=0)

Time tags for ERA5 files ...
202106
2021060100_2021060105
tzc
Serial read of data 
Reading data vars for 2021060100_2021060105 took  0.2666 seconds
Extracting values from Xarrays took 106.2300 seconds
ERA5 Global mean surface pressure= 79216312483.85638
ERA5 Global mean surface pressure= 79214225189.45654
ERA5 Global mean surface pressure= 79211889567.42714
ERA5 Global mean surface pressure= 79209594214.74307
ERA5 Global mean surface pressure= 79208043844.9477
ERA5 Global mean surface pressure= 79206909591.72078
shape of a_model  (137,)
Reading one set of ERA5 vars took  106.9020 seconds


In [4]:
fii=GR.xRegrid(HorzInterpLnPs=False)

starting xRegrid ERA5 _x_ Arctic at Wed Jul 19 16:01:45 2023 
Finished phis Horz Rgrd  0.2876 seconds
Finished ps Horz Rgrd  0.3681 seconds
Finished te_ERA Horz Rgrd  42.6914 seconds
Finding Te150 and P150 took  7.3601 seconds
 In PsAdjust 
PHIS (117398,)
PHIS_CAM (117398,)
PS (6, 117398)
Creating 3D P-fields etc., took   7.7876 seconds
WilliamsonOlson surface 
Williamson Olson surface took  19.3182 seconds
 going into vertical regrid of T 
Flexible VertRegrid using reshaped ARRAYS 
nworkers available  1
Using serial code which is faster for 1 worker
Pll'zd Vertical int 124.0357 seconds
 going into horz+vertical regrid of Q 
Horz RG in fullRegrid 
Vert RG in fullRegrid 
Flexible VertRegrid using reshaped ARRAYS 
nworkers available  1
Using serial code which is faster for 1 worker
Pll'zd Vertical int 49.7282 seconds
 ... Bottom filling took 5.8229 seconds
 going into horz+vertical regrid of U 
Horz RG in fullRegrid 
Vert RG in fullRegrid 
Flexible VertRegrid using reshaped ARRAYS 
nwork

In [5]:
#importlib.reload( WrDst )


In [6]:
fuu=WrDst.write_netcdf()

6
/glade/scratch/juliob/ERA5_x_Arctic_L32__WO.2021-06-01-00000.nc
/glade/scratch/juliob/ERA5_x_Arctic_L32__WO.2021-06-01-03600.nc
/glade/scratch/juliob/ERA5_x_Arctic_L32__WO.2021-06-01-07200.nc
/glade/scratch/juliob/ERA5_x_Arctic_L32__WO.2021-06-01-10800.nc
/glade/scratch/juliob/ERA5_x_Arctic_L32__WO.2021-06-01-14400.nc
/glade/scratch/juliob/ERA5_x_Arctic_L32__WO.2021-06-01-18000.nc


In [ ]:
#STOPP 

In [ ]:
from GlobalVarClass import Gv
print(Gv.dstTZHkey)

In [ ]:
#fuu=GR.write_netcdf(version='test02-oldtopo')

In [ ]:
# Let's look at some interensitng points here
#
location="SEPac"

if (location=="Himalayas"):
    #Himalayan peaks
    Plat,Plon = 36.,76.
    ylim=(60_000.,42_000.)
    xlim=(240,270 )

if (location=="Sahara"):
    #Bordj Badji Mokhtar Algeria
    Plat,Plon = 23.9,-1.8
    ylim=(100_000.,80_000.)
    xlim=(290,330 )

if (location=="Sonora"):
    #Caborca
    Plat,Plon = 30.7,-112.1
    ylim=(100_000.,80_000.)
    xlim=(295,315 )


if (location=="Andes"):
    #Andean peaks
    Plat,Plon=-30.,290.
    ylim=(85_000.,60_000.)
    xlim=(250,300 )

if (location=="SEPac"):
    #
    Plat,Plon=-30.,250.
    ylim=(100_000.,60_000.)
    xlim=(250,300 )

if (location=="Australia"):
    #Australia desert (Alice Springs)
    Plat,Plon=-23.4,135.
    ylim=(96_000.,85_000.)
    xlim=(280,320 )

if (Plon<0.):
    Plon=Plon+360.
    
if (Gv.dstTZHkey=='tzyx'):
    iCAM=np.argmin( np.abs( Gv.lon_CAM - Plon ))
    jCAM=np.argmin( np.abs( Gv.lat_CAM - Plat ))
if (Gv.dstTZHkey=='tzc'):
    cCAM=np.argmin( np.sqrt(  (Gv.lon_CAM - Plon)**2 +  (Gv.lat_CAM - Plat)**2      )      )

iERA=np.argmin( np.abs( Gv.lon_ERA - Plon ))
jERA=np.argmin( np.abs( Gv.lat_ERA - Plat ))
print(cCAM)
print(Gv.lon_CAM[cCAM],Gv.lat_CAM[cCAM])
plt.plot(Gv.lon_ERA)

In [ ]:
#print(te_ERA_xzCAM[nCAM,kCAM,ilat,ilon] )

"""
plt.plot( te_ERA_xCAM [nCAM,:, jCAM, iCAM] , pmid_CAM_zERA [nCAM,:, jCAM, iCAM], '-x' 
         , label='ERA Temp from Python code on xCAM and zERA' )


plt.plot( te_ERA_xzCAM [nCAM,:, jCAM, iCAM], pmid_CAM [nCAM,:, jCAM, iCAM], '-^' 
         , label='ERA Temp from Python code on xzCAM' )
"""

for ntime in np.arange(24):
    if (ntime==0):
        label1='ERA Temps'
        label2='ERA Temps on xCAM'
    else:
        label1=None
        label2=None
        
    plt.plot( Gv.te_ERA [ntime,:, jERA, iERA] , Gv.pmid_ERA [ntime,:, jERA, iERA], color='red'
            , label=label1 )

    plt.plot( Gv.te_ERA_xzCAM [ntime,:, cCAM] , Gv.pmid_CAM [ntime,:, cCAM], color='orange'
            , label=label2 )


#plt.ylim( ylim )
#plt.xlim( xlim )
plt.ylim( 103_000.,90_000.)
plt.xlim( 285.,295. )


plt.legend(loc='upper left')

In [ ]:
print(np.shape(Gv.pmid_CAM_zERA))#_zERA))
plt.plot(Gv.pmid_CAM_zERA[0,:,20000])

In [ ]:
clev=np.linspace(200,330,num=31)
clev=np.linspace(200,290,num=31)
dlev=np.linspace(-10,10,num=21)
#clev=np.linspace(277,287,num=31)
#ERA - 89 128
#CAM -124 72
print(Gv.lon_ERA[128], Gv.lat_ERA[89])

# Global
elon,wlon=0,360
slat,nlat=-90,90

# Tibet ...
#elon,wlon=70,100
#slat,nlat=25,40

# Andes ...
#elon,wlon=280,300
#slat,nlat=-40,-10

# Himalayan Front
#elon,wlon=85,95
#slat,nlat=25,28


kCAM=57
kERA=136
kCAM=10
kERA=90
nCAM=0

fig,ax = plt.subplots(2,2,figsize=(18,18) ,squeeze=False )

i,j=0,0
cf = ax[i,j].tricontourf( Gv.lon_CAM, Gv.lat_CAM, Gv.te_ERA_xCAM[nCAM,kERA,:],levels=clev )
ax[i,j].set_xlim(elon,wlon)
ax[i,j].set_ylim(slat,nlat)
ax[i,j].set_title("ERA Temp on xzCAM grid from Python code")
plt.colorbar(cf,ax=ax[i,j] )

i,j=0,1
cf = ax[i,j].contourf( Gv.lon_ERA, Gv.lat_ERA, Gv.te_ERA[nCAM,kERA,:,:],levels=clev )
ax[i,j].set_xlim(elon,wlon)
ax[i,j].set_ylim(slat,nlat)
ax[i,j].set_title("ERA Temp on ERA xz grid")
plt.colorbar(cf,ax=ax[i,j] )


In [ ]:
clev=np.linspace(200,330,num=31)
dlev=np.linspace(-10,10,num=21)
#clev=np.linspace(277,287,num=31)
#ERA - 89 128
#CAM -124 72
print(Gv.lon_ERA[128], Gv.lat_ERA[89])

# Global
elon,wlon=0,360
slat,nlat=-90,90

# Tibet ...
#elon,wlon=70,100
#slat,nlat=25,40

# Andes ...
#elon,wlon=280,300
#slat,nlat=-40,-10

# Himalayan Front
#elon,wlon=85,95
#slat,nlat=25,28


kCAM=57
kERA=136
nCAM=5

fig,ax = plt.subplots(2,2,figsize=(18,18) ,squeeze=False )

i,j=0,0
cf = ax[i,j].contourf( Gv.lon_CAM, Gv.lat_CAM, Gv.te_ERA_xzCAM[nCAM,kCAM,:,:],levels=clev )
ax[i,j].set_xlim(elon,wlon)
ax[i,j].set_ylim(slat,nlat)
ax[i,j].set_title("ERA Temp on xzCAM grid from Python code")
plt.colorbar(cf,ax=ax[i,j] )

i,j=0,1
cf = ax[i,j].contourf( Gv.lon_CAM, Gv.lat_CAM, Gv.te_ERA_xCAM[nCAM,kERA,:,:],levels=clev )
ax[i,j].set_xlim(elon,wlon)
ax[i,j].set_ylim(slat,nlat)
ax[i,j].set_title("ERA Temp on xCAM/zERA grid")
plt.colorbar(cf,ax=ax[i,j] )


In [ ]:
print( np.minimum( [0.],[200])[0] )

In [ ]:
Dphis = phis_ERA_xCAM - phis_CAM
plt.plot(Dphis.flatten()-dphis_WO.flatten() )

In [ ]:
file1='/glade/scratch/juliob/ERA5_x_fv1x1_L58_poo4_WO.2017-01-01-18000.nc'
dood=xr.open_dataset( file1 )

## Check Mapping files

In [ ]:


#ERA - 89 128
#CAM -124 72
jCAM,iCAM = 124,72
jERA,iERA = 89,128
nCAM = 3

plt.plot( te_ERA_xCAM [nCAM,:, jCAM, iCAM] , pmid_CAM_zERA [nCAM,:, jCAM, iCAM], '-x' 
         , label='ERA Temp from Python code on xCAM and zERA' )

plt.plot( te_ERA_xCAM_wo2[:, jCAM, iCAM] , press_ERA[:, jCAM, iCAM], '-o' 
         , label='ERA Temp on xCAM and zERA from GUTS OF W&O' )



#plt.plot( te_ERA_xzCAM [0,:,124, 72] , pmid_CAM [0,:,124, 72], '-o' , label='ERA Temp from Python code on xzCAM' )
#plt.plot( te_ERA[0,:,89, 128] ,pmid_ERA [0,:,89, 128] ,'-+', label='ERA Temp at closest xERA point' )


plt.ylim(88_000.,70_000.)
plt.xlim(270,290)
plt.legend()

print(ps_new[jCAM,iCAM])
print(ps_CAM[nCAM,jCAM,iCAM])

print(ps_old[jCAM,iCAM])
print(ps_ERA_xCAM[nCAM,jCAM,iCAM])

